In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.types import Send
from langgraph.graph import StateGraph
from langgraph.graph import START, END
from langchain_core.runnables import RunnableConfig
from llm import ollama_model, watsonx_model
from state import (
    OverallState,
    QueryGenerationState,
    ReflectionState,
    WebSearchState,
)
from configuration import Configuration
llm = ollama_model

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///./database/app.db")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM users LIMIT 5;")}')

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

In [ ]:
from typing import Literal
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode



get_schema_tool = next(tool for tool in tools if tool.name == "sql_db_schema")
get_schema_node = ToolNode([get_schema_tool], name="get_schema")

run_query_tool = next(tool for tool in tools if tool.name == "sql_db_query")
run_query_node = ToolNode([run_query_tool], name="run_query")


# Example: create a predetermined tool call
def list_tables(state: MessagesState):
    tool_call = {
        "name": "sql_db_list_tables",
        "args": {},
        "id": "abc123",
        "type": "tool_call",
    }
    tool_call_message = AIMessage(content="", tool_calls=[tool_call])

    list_tables_tool = next(tool for tool in tools if tool.name == "sql_db_list_tables")
    tool_message = list_tables_tool.invoke(tool_call)
    response = AIMessage(f"Available tables: {tool_message.content}")

    return {"messages": [tool_call_message, tool_message, response]}


# Example: force a model to create a tool call
def call_get_schema(state: MessagesState):
    # Note that LangChain enforces that all models accept `tool_choice="any"`
    # as well as `tool_choice=<string name of tool>`.
    llm_with_tools = llm.bind_tools([get_schema_tool], tool_choice="auto")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}





def generate_query(state: MessagesState):
    generate_query_system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

{input_question}
""".format(
    dialect=db.dialect,
    top_k=5,
    input_question=state["messages"][-1].content,
)
    user_message = {
        "role": "user",
        "content": generate_query_system_prompt,
    }
    # We do not force a tool call here, to allow the model to
    # respond naturally when it obtains the solution.
    llm_with_tools = llm.bind_tools([run_query_tool])
    response = llm_with_tools.invoke([user_message])
    return {"messages": [response]}


check_query_system_prompt = """
You are a SQL expert with a strong attention to detail.
Double check the {dialect} query for common mistakes, including:
    - Using NOT IN with NULL values
    - Using UNION when UNION ALL should have been used
    - Using BETWEEN for exclusive ranges
    - Data type mismatch in predicates
    - Properly quoting identifiers
    - Using the correct number of arguments for functions
    - Casting to the correct data type
    - Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes,
just reproduce the original query.

You will call the appropriate tool to execute the query after running this check.
""".format(dialect=db.dialect)


def check_query(state: MessagesState):
    system_message = {
        "role": "system",
        "content": check_query_system_prompt,
    }

    # Generate an artificial user message to check
    tool_call = state["messages"][-1].tool_calls[0]
    user_message = {"role": "user", "content": tool_call["args"]["query"]}
    llm_with_tools = llm.bind_tools([run_query_tool], tool_choice="auto")
    response = llm_with_tools.invoke([system_message, user_message])
    response.id = state["messages"][-1].id
    return {"messages": [response]}

# State (The data that gets pass between nodes)

In [10]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import TypedDict

from langgraph.graph import add_messages
from typing_extensions import Annotated


import operator


class OverallState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: int 
    firstName: str
    lastName: str
    subscriptions: Annotated[list, operator.add]
    # search_query: Annotated[list, operator.add]
    # web_research_result: Annotated[list, operator.add]
    # sources_gathered: Annotated[list, operator.add]
    # initial_search_query_count: int
    # max_research_loops: int
    # research_loop_count: int
    # reasoning_model: str

# Get specific user's subscription information when recieve the User_ID and questions

In [ ]:
from database.database import SessionLocal
from sqlalchemy import text
db = SessionLocal()
user_id = 1
out = db.execute(text(f"""SELECT 
    u.id as user_id,
    u.email,
    u.firstName,
    u.lastName,
    s.start_date as subscription_start,
    s.current_status as subscription_status,
    st.tier_name,
    st.price_usd,
    sv.name as service_name,
    sv.category
FROM users u
LEFT JOIN subscriptions s ON u.id = s.user_id
LEFT JOIN service_tiers st ON s.tier_id = st.id
LEFT JOIN services sv ON st.service_id = sv.id
WHERE u.id = {user_id};"""))

rows_as_dicts = [row._asdict() for row in out]

for row_dict in rows_as_dicts:
    print(row_dict)



## Node Setup

In [ ]:
from database.database import SessionLocal
from sqlalchemy import text


def get_user_subscription(state: OverallState):
    # configurable = Configuration.from_runnable_config(config)

    db = SessionLocal()
    user_id = state["user_id"]
    out = db.execute(text(f"""SELECT 
    s.start_date as subscription_start,
    s.current_status as subscription_status,
    st.tier_name,
    st.price_usd,
    sv.name as service_name,
    sv.category
    FROM users u
    LEFT JOIN subscriptions s ON u.id = s.user_id
    LEFT JOIN service_tiers st ON s.tier_id = st.id
    LEFT JOIN services sv ON st.service_id = sv.id
    WHERE u.id = {user_id};"""))

    subscriptions = []
    rows_as_dicts = [row._asdict() for row in out]
    for row_dict in rows_as_dicts:
        subscriptions.append(row_dict)

    out = db.execute(text(f"""SELECT 
    u.firstName,
    u.lastName,
    FROM users u
    WHERE u.id = {user_id};"""))
    out = [row._asdict() for row in out]
    firstName = out[0]["firstName"]
    lastName = out[0]["lastName"]
    
    return {'subscriptions': subscriptions, "firstName": firstName, "lastName": lastName}


### Testing the Node

In [ ]:
from langchain_core.messages import HumanMessage

test_state = {
    
    'messages': [
        HumanMessage(content="Test message")
    ],
    'user_id': 1
}
out = get_user_subscription(test_state)

print(out)


### testing other joined tables

In [ ]:
from database.database import SessionLocal
from sqlalchemy import text
db = SessionLocal()
user_id = 1
out = db.execute(text(f"""SELECT 
    u.email,
    u.firstName,
    u.lastName,
    s.start_date,
    s.next_renewal_date,
    s.amount_billed,
    s.currency,
    st.tier_name,
    st.billing_period,
    sv.name as service_name,
    sv.website
FROM users u
LEFT JOIN subscriptions s ON u.id = s.user_id
LEFT JOIN service_tiers st ON s.tier_id = st.id
LEFT JOIN services sv ON st.service_id = sv.id
WHERE u.id = {user_id}
ORDER BY u.lastName, u.firstName;"""))


for row in out: 
    print(row)


# Prompt to allow Agent to view the user subscriptions

## Node Setup

In [22]:
from database.database import SessionLocal
from sqlalchemy import text


def agent_review_user_subscription(state: OverallState):
    # configurable = Configuration.from_runnable_config(config)
    firstName = state["firstName"]
    question = state["messages"][0].content
    subscription_history = state["subscriptions"]

    column = "service_name | category | tier_name | price_usd | subscription_start | subscription_status\n"
    format_subscription_history = column + "\n".join([f"{sub['service_name']} | {sub['category']} | {sub['tier_name']} | {sub['price_usd']} | {sub['subscription_start']} | {sub['subscription_status']}" for sub in subscription_history])
    # print(format_subscription_history)
    subscription_analysis_prompt = f"""
You are a helpful financial assistant. Your job is to analyze the user's subscription history and recommend ways to reduce their monthly spending.
Here is the user's subscription history:
    {format_subscription_history}

Based on the provided subscription data, do the following:
    1.	Usage Analysis

    •	Determine if the user is actively using each subscription (based on any usage data if available).
    •	If usage is not provided, don't make assumptions.

    2.	Cost Efficiency Evaluation

    •	Identify the most expensive subscriptions.
    •	Determine whether each subscription appears to offer good value for the price.

    3.	Cheaper Alternatives

    •	Search online for cheaper alternatives with similar features (e.g., if the user pays for Spotify, check for cheaper music services or free versions).
    •	Provide relevant names and estimated prices of those services.

    4.	Cancellation Suggestions

    •	If a service is rarely or never used, or is overpriced compared to alternatives, suggest canceling it.

    5.	Summary of Actions

    •	Summarize in clear, actionable steps:
    •	Which subscriptions to keep, cancel, or replace (and with what).
    •	Total estimated monthly savings if recommendations are followed.

When generating your response:
    •	Be concise but informative.
    •	Use bullet points for clarity.
    •	Be neutral and helpful, DO NOT shame the user for spending.
    •	Use the user name to respond to the user casually: {firstName}. 
    •	At the end, please offer user the recommandation actions that you have suggested.

    Question: {question}
    """    

    # if "ibm" in configurable.subscription_analysis_model:
    #     llm = watsonx_model
    # else:
    #     llm = ollama_model

    llm = watsonx_model
    message = llm.invoke(subscription_analysis_prompt)

    return {'messages': message, 'subscription_suggestion': message}


In [23]:
test_state = {
    'messages': [
        HumanMessage(content="How can I manage my budget on my subscriptions on Netflix?")
    ],
    'user_id': 1, 
    'firstName': 'Alice', 
    'lastName': 'Smith',
    'subscriptions': [
        {'subscription_start': '2023-01-08', 'subscription_status': 'cancelled', 'tier_name': 'Standard with Ads', 'price_usd': 6.99, 'service_name': 'Netflix', 'category': 'Streaming'}, 
        {'subscription_start': '2023-04-01', 'subscription_status': 'active', 'tier_name': 'Monthly', 'price_usd': 14.99, 'service_name': 'Amazon Prime', 'category': 'Shopping'}, 
        {'subscription_start': '2023-07-01', 'subscription_status': 'active', 'tier_name': 'All Apps', 'price_usd': 54.99, 'service_name': 'Adobe Creative Cloud', 'category': 'Productivity'}
    ]
}

out = agent_review_user_subscription(test_state)

print(out['messages'].content)



Hello Alice,

Here's a breakdown of your current subscriptions and recommendations to help you manage your budget:

1. **Usage Analysis**
   - **Netflix (Standard with Ads)**: Canceled on 2023-01-08, so it's unlikely you're currently using this service.
   - **Amazon Prime**: Active, but usage depends on shopping and potentially Prime Video/Music.
   - **Adobe Creative Cloud**: Active, indicating regular use.

2. **Cost Efficiency Evaluation**
   - **Most Expensive**: Adobe Creative Cloud at $54.99/month.
   - **Value for Price**: Without usage data, we can't definitively say if these subscriptions offer good value. However, Adobe Creative Cloud is often considered valuable for professionals.

3. **Cheaper Alternatives**
   - **Streaming (Netflix)**: Cheaper alternatives include Disney+ ($7.99/month), Hulu with Ads ($6.99/month), or Paramount+ ($4.99/month).
   - **Shopping (Amazon Prime)**: Consider Walmart+ ($12.95/year or $9.95/month), which offers free 2-day shipping and other perk

## Test output 

1. Usage Analysis:
   - Netflix | Streaming | Standard with Ads | $6.99 | 2023-01-08 | cancelled: Given that the subscription status is 'cancelled', Alice is no longer actively using Netflix.
   - Amazon Prime | Shopping | Monthly | $14.99 | 2023-04-01 | active: As the status is 'active', Alice is currently using Amazon Prime.
   - Adobe Creative Cloud | Productivity | All Apps | $54.99 | 2023-07-01 | active: With an 'active' status, Alice is using Adobe Creative Cloud.

2. Cost Efficiency Evaluation:
   - The most expensive subscription is Adobe Creative Cloud at $54.99.
   - It appears that Amazon Prime at $14.99 offers a reasonable value given its shopping benefits beyond streaming.
   - Netflix, though cancelled, was priced reasonably for streaming considering its standard ad-supported tier.

3. Cheaper Alternatives:
   - Streaming: Consider Disney+ ($7.99/month) or Hulu ($6.99/month with ads) for movie and TV content similar to Netflix.
   - Productivity: Explore alternatives like Canva (free basic plan, paid plans starting at $12.99/month for additional features) or GIMP (free) for graphic design.

4. Cancellation Suggestions:
   - Since Alice has already cancelled Netflix, no action is needed there.
   - For Adobe Creative Cloud, if Alice doesn't frequently use professional design software, consider switching to a cheaper plan like 'Photography' ($9.99/month) or exploring free/cheaper alternatives for occasional use.

5. Summary of Actions:
   - Keep Amazon Prime as it seems to provide value with shopping benefits and possibly streaming.
   - Evaluate Adobe Creative Cloud usage:
     - If infrequent, switch to a less expensive plan (Photography plan).
     - If occasional, consider Canva for free basic design needs or GIMP as a fully free alternative.
   - No action needed for Netflix (already cancelled).
   - Total estimated monthly savings: If Alice switches Adobe Creative Cloud to the Photography plan, she could save around $35 monthly, and further savings could be made by transitioning to Canva or GIMP for specific design tasks.

Remember, Alice, these are suggestions tailored to your current subscriptions; personal preferences and needs should guide your final decisions.

In [ ]:
def should_continue(state: MessagesState) -> Literal[END, "check_query"]:
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return END
    else:
        return "check_query"


builder = StateGraph(MessagesState)
builder.add_node(list_tables)
builder.add_node(call_get_schema)
builder.add_node(get_schema_node, "get_schema")
builder.add_node(generate_query)
builder.add_node(check_query)
builder.add_node(run_query_node, "run_query")

builder.add_edge(START, "list_tables")
builder.add_edge("list_tables", "call_get_schema")
builder.add_edge("call_get_schema", "get_schema")
builder.add_edge("get_schema", "generate_query")
builder.add_conditional_edges(
    "generate_query",
    should_continue,
)
builder.add_edge("check_query", "run_query")
builder.add_edge("run_query", "generate_query")

agent = builder.compile()

## Showing the flow graph of the agent

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(Image(agent.get_graph().draw_mermaid_png()))


## Testing Agent

In [ ]:
question = "What is the most expensive subscriptions and their names that Bob is subscribed to and what is the name of that service?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="debug",
):
    print(step)
    # step["messages"][-1].pretty_print()
